In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('stats.csv')

## Calculating Best Global Heroes

In [ ]:
class best_Heroes:
    def __init__(self, data):
        self.data = data
        self.hero_stats = {}
        self._populate_hero_stats(mirrored = True)
        self.hero_categories = [
            {"name": "Ana", "category": ["Poke", "Brawl", "Dive"]},
            {"name": "Ashe", "category": ["Poke", "Brawl"]},
            {"name": "Bap", "category": ["Brawl", "Poke"]},
            {"name": "Bastion", "category": ["Poke", "Brawl"]},
            {"name": "Brig", "category": ["Dive"]},
            {"name": "Cass", "category": ["Brawl"]},
            {"name": "DVA", "category": ["Brawl", "Dive"]},
            {"name": "Doom", "category": ["Brawl", "Dive"]},
            {"name": "Echo", "category": ["Dive", "Brawl"]},
            {"name": "Genji", "category": ["Dive", "Brawl", "Poke"]},
            {"name": "Hanzo", "category": ["Poke"]},
            {"name": "Hazard", "category": ["Brawl"]},  
            {"name": "Illari", "category": ["Poke"]},  
            {"name": "Queen", "category": ["Brawl"]},
            {"name": "Junk", "category": ["Brawl"]},
            {"name": "Juno", "category": ["Dive", "Brawl"]},  
            {"name": "Kiri", "category": ["Dive", "Poke", "Brawl"]}, 
            {"name": "LW", "category": ["Dive"]},
            {"name": "Lucio", "category": ["Brawl", "Dive"]}, 
            {"name": "Mauga", "category": ["Brawl"]},
            {"name": "Mei", "category": ["Brawl", "Poke"]}, 
            {"name": "Mercy", "category": ["Poke"]},
            {"name": "Moira", "category": ["Dive", "Brawl"]},
            {"name": "Orisa", "category": ["Brawl", "Poke"]},
            {"name": "Pharah", "category": ["Poke"]},
            {"name": "Ram", "category": ["Brawl", "Poke"]},
            {"name": "Reaper", "category": ["Brawl", "Dive"]},
            {"name": "Rein", "category": ["Brawl"]},
            {"name": "Hog", "category": ["Poke"]},
            {"name": "Sigma", "category": ["Poke", "Brawl"]},
            {"name": "Soj", "category": ["Poke"]},  
            {"name": "Soldier", "category": ["Poke"]},
            {"name": "Sombra", "category": ["Dive"]},
            {"name": "Sym", "category": ["Brawl"]},
            {"name": "Torb", "category": ["Poke", "Brawl"]},  
            {"name": "Tracer", "category": ["Dive"]}, 
            {"name": "Venture", "category": ["Dive", "Brawl"]},  
            {"name": "Widow", "category": ["Poke"]},  
            {"name": "Winston", "category": ["Dive"]},
            {"name": "Ball", "category": ["Dive"]}, 
            {"name": "Zarya", "category": ["Brawl"]},  
            {"name": "Zen", "category": ["Poke"]}  
            ]
    def _populate_hero_stats(self, mirrored):
        # Iterate through the rows of the data to populate hero statistics
        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')


            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates
            
            for hero in all_heroes:
                if not mirrored and hero in allied_heroes and hero in enemy_heroes:
                    continue  

                if hero not in self.hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    self.hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                if hero in allied_heroes:
                    self.hero_stats[hero]['matches'] += 1
                    if result == 1:
                        self.hero_stats[hero]['wins'] += 1
                if hero in enemy_heroes:
                    self.hero_stats[hero]['matches'] += 1
                    if result == 0:
                        self.hero_stats[hero]['wins'] += 1
                
        # remove heroes with no matches
        self.hero_stats = {hero: stats for hero, stats in self.hero_stats.items() if stats['matches'] > 0}

    def get_bayesian_winrate(self, match_wins, a=25, b=25):
        bayesian_winrates: dict[str, list[int]] = {}

        for hero in match_wins:
            matches = match_wins[hero]['matches']
            wins = match_wins[hero]['wins']
            bayesian_winrates[hero] = (wins + a) / (matches + a + b)  # Posterior mean of winrate
        
        return bayesian_winrates
    


    def normalize_winrates(self, winrates):
        """Normalises the winrates across roles. 
        
        - Requires winrate
        
        
        """
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        # Separate winrates by role
        for hero, winrate in winrates.items():
            role = self.hero_stats[hero]['Role']
            roles[role].append(winrate)
        
        normalized = {}
        
        # Normalize winrates within each role
        for role, winrate_list in roles.items():
            winrate_array = np.array(winrate_list)
            min_winrate = np.min(winrate_array)
            max_winrate = np.max(winrate_array)
            
            for hero, winrate in winrates.items():
                if self.hero_stats[hero]['Role'] == role:
                    normalized[hero] = (winrate - min_winrate) / (max_winrate - min_winrate)
        
        return normalized

    def stat_visualiser(self, values):
        # Separate heroes by role
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        for hero, value in values.items():
            role = self.hero_stats[hero]['Role']
            if value >= 0.75:
                roles[role].append((hero, value))
        
        # Sort each role by value in descending order
        for role in roles:
            roles[role].sort(key=lambda x: x[1], reverse=True)
        
        # Print header
        print(f"{'Tank':<20} {'DPS':<20} {'Support':<20}")
        print("-" * 60)
        
        # Print heroes side by side
        for i in range(max(len(roles['Tank']), len(roles['Support']), len(roles['DPS']))):
            tank = f"{roles['Tank'][i][0]}: {roles['Tank'][i][1]:.4f}" if i < len(roles['Tank']) else ""
            dps = f"{roles['DPS'][i][0]}: {roles['DPS'][i][1]:.4f}" if i < len(roles['DPS']) else ""
            support = f"{roles['Support'][i][0]}: {roles['Support'][i][1]:.4f}" if i < len(roles['Support']) else ""            
            print(f"{tank:<20} {dps:<20} {support:<20}")


    def first_pick(self, teammates, curr_map, comp = "Poke,Brawl,Dive"):
        """ Calculate the total matches and wins of all heroes that were played on a given map or with specific teammates"""

        comp_categories = set(comp.split(","))
        teammates = [hero.strip() for hero in teammates.split(',')] if isinstance(teammates, str) else teammates
        hero_stats: dict[str, list[int]] = {}

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',')
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')
            my_map = row['Map']

            # Check if any teammate is in allied heroes or if the map matches the current map
            self.synergy(teammates, curr_map, hero_stats, row, result, allied_heroes, enemy_heroes, my_map, comp_categories)


        for teammate in teammates:
            if teammate in hero_stats:
                del hero_stats[teammate]
        return hero_stats

    def synergy(self, teammates, curr_map, hero_stats, row, result, allied_heroes, enemy_heroes, my_map, comp_categories):
        if any(teammate in allied_heroes for teammate in teammates) or my_map == curr_map:
            for hero in allied_heroes:
                
                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue
                if hero not in hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}
                
                hero_stats[hero]['matches'] += 1
                if result == 1:
                    hero_stats[hero]['wins'] += 1

        if any(enemy in enemy_heroes for enemy in teammates) or my_map == curr_map:
            for hero in enemy_heroes:

                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue

                if hero not in hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}
                hero_stats[hero]['matches'] += 1
                if result == 0:
                    hero_stats[hero]['wins'] += 1
                
heroes = best_Heroes(df)
bayesian_winrates = heroes.get_bayesian_winrate(heroes.hero_stats)
normalised_winrates = heroes.normalize_winrates(bayesian_winrates)
heroes.stat_visualiser(normalised_winrates)



Tank                 DPS                  Support             
------------------------------------------------------------
Winston: 1.0000      Torb: 1.0000         Moira: 1.0000       
Ball: 0.9838         Genji: 0.8288                            
                     Sym: 0.8132                              
